# Equalização

Neste notebook iremos explorar os próximos blocos na cadeia de algoritmos DSP que são dois equalizadores lineares que compensam os efeitos da dispersão cromática (CD) e da dispersão do modo de polarização (PMD).

Antes do renascimento da detecção coerente, o CD em sistemas ópticos com modulação de intensidade e detecção direta era controlado pela inserção de módulos de compensação de dispersão óptica ao longo do link óptico. O manejo da CD foi um tema relevante e envolveu a interdependência da DC com efeitos não lineares. Em sistemas ópticos digitais coerentes, entretanto, a compensação CD é completamente deixada para o domínio digital. As vantagens dessa escolha são diversas, como a redução das perdas de inserção causadas pelos módulos ópticos de compensação de CD e a facilitação do projeto e gerenciamento do sistema.

#### Equalização estática (compensação CD):

Nos receptores ópticos digitais coerentes, o CD é compensado por filtros estáticos aplicados separadamente aos sinais das duas orientações de polarização. Como CD é praticamente invariante no tempo e independente da polarização, os filtros correspondentes às duas orientações de polarização possuem os mesmos coeficientes.